In [5]:
import pandas as pd

In [6]:
position_option = pd.read_csv('./data/position_option.csv').drop_duplicates()
position_option = position_option[["time", "symbol", "contractRight", "lastTradeDateOrContractMonth", "strike", "delta"]]

In [7]:
# get the latest record with unique symbol secType contractRight lastTradeDateOrContractMonth strike

comparator = position_option.groupby(["symbol", "contractRight"]).agg({'time': 'max'}).reset_index()

# get the delta of the latest record with unique symbol secType contractRight lastTradeDateOrContractMonth strike

order_calculator = position_option.merge(comparator, on=['symbol', "contractRight",'time'], how='inner')

# sum the delta of same symbol

order_calculator.groupby(['symbol']).agg({'delta': 'sum'}).reset_index()





,symbol,delta
0,NFLX,-0.040545


In [8]:
order_calculator

,time,symbol,contractRight,delta
0,2023-10-04 09:37:14,NFLX,P,-0.507811
1,2023-10-04 09:37:14,NFLX,P,-0.508558
2,2023-10-04 09:37:14,NFLX,P,-0.507467
3,2023-10-04 09:37:14,NFLX,P,NaN
4,2023-10-04 09:37:15,NFLX,C,0.494398
5,2023-10-04 09:37:15,NFLX,C,0.493856
6,2023-10-04 09:37:15,NFLX,C,0.495037
